In [22]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import os
import sys
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, grandparent_dir, "src", "utils")
sys.path.append(utils_dir)
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, "src", "utils")
sys.path.append(utils_dir)

In [23]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

from process_data import process_data
from weighted_accuracy import weighted_accuracy_scorer
from plot_learning_curves import plot_learning_curves
from weighted_accuracy_and_tools import decompose_y, reconstruct_y


In [24]:
X = os.path.join("..","data","original_input", "X_train_filled.csv")
Y = os.path.join("..","data","original_input", "X_test_GgyECq8.csv")
W = os.path.join("..","data","original_input", "y_train_jJtXgMX.csv")

X_train = pd.read_csv(X, delimiter=',')
X_test = pd.read_csv(Y,delimiter=',')
y_train = pd.read_csv(W, delimiter=',')
delivery_start = X_test["DELIVERY_START"]

In [25]:
X_train = process_data(X_train.copy(deep=True), "predicted_spot_price", None, "standard")
X_test = process_data(X_test.copy(deep=True), "predicted_spot_price", None, "standard")
y_train = process_data(y_train.copy(deep=True), None, None, None)

In [26]:
y_train_direction, y_train_magnitude = decompose_y(y_train['spot_id_delta'])

In [27]:
#from sklearn.ensemble import RandomForestRegressor
#from sklearn.model_selection import GridSearchCV
#
## Initialize the RandomForestRegressor
#reg = RandomForestRegressor(n_jobs=-1, **{'min_samples_leaf': 2, 'min_samples_split': 5})
#
## Set up the parameter grid to search
#reg.fit(X_train, y_train_magnitude)
#y_pred_magn = reg.predict(X_test)

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming X_train, X_test, y_train_magnitude, y_test_magnitude are already defined

# Initialize the Linear Regression model
lr = LinearRegression()

# Initialize Sequential Feature Selector
sfs = SequentialFeatureSelector(lr, n_features_to_select="auto", direction='forward', scoring='neg_mean_squared_error', cv=5)

# Fit the Sequential Feature Selector with the training data
sfs.fit(X_train, y_train_magnitude)

# Transform training and test sets so they only contain the selected features
X_train_selected = sfs.transform(X_train)
X_test_selected = sfs.transform(X_test)

# Fit the Linear Regression model on the selected features
lr.fit(X_train_selected, y_train_magnitude)

# Make predictions on the test set using the selected features
y_pred_magn = lr.predict(X_test_selected)

# Calculate the Mean Squared Error for the predictions
#mse = mean_squared_error(y_test_magnitude, y_pred_magn)
#print(f'Mean Squared Error: {mse}')

# To see which features were selected
selected_features = np.where(sfs.get_support())[0]
print(f'Selected features (0-indexed): {selected_features}')


Selected features (0-indexed): [3 4 6 7]


In [29]:
d = {"DELIVERY_START" : delivery_start, "spot_id_delta" : y_pred_magn.reshape(-1)}
y_pred = pd.DataFrame(data = d)

y_pred.to_csv("../data/output_y_test/y_result_magn_rdf1.csv", index = False)

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Initialize and train the regressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42, **{'min_samples_leaf': 4, 'min_samples_split': 10})
regressor.fit(X_train, y_train_direction)

RandomForestRegressor(min_samples_leaf=4, min_samples_split=10, random_state=42)

In [31]:
# Predict using the regressor
predictions = regressor.predict(X_test)

# Apply thresholds to classify predictions
# Below 0.3 will be class 0, above 0.7 will be class 1, and in between will be unclassified (-1)
classified_predictions = []
for pred in predictions:
    if pred < 0.30:
        classified_predictions.append(0)
    elif pred > 0.35:
        classified_predictions.append(1)
    else:
        classified_predictions.append(-1)  # Unclassified
from collections import Counter

# classified_predictions contains the classified results, including -1 for unclassified instances # This should be the list you obtained in the previous step

# Use Counter to tally the occurrences of each class
class_counts = Counter(classified_predictions)

# Print the count of each class
print("Class 0 count:", class_counts[0])
print("Class 1 count:", class_counts[1])
print("Unclassified count:", class_counts[-1])


Class 0 count: 520
Class 1 count: 4141
Unclassified count: 281


In [32]:
y_pred_dir = np.array(classified_predictions)

# Reassign unclassified (-1) predictions to class 1
y_pred_dir[y_pred_dir == -1] = 1

In [33]:
y_pred = reconstruct_y(y_pred_dir, y_pred_magn)

In [34]:
delivery_start, y_pred

(0       2023-04-02 00:00:00+02:00
 1       2023-04-02 01:00:00+02:00
 2       2023-04-02 02:00:00+02:00
 3       2023-04-02 03:00:00+02:00
 4       2023-04-02 04:00:00+02:00
                   ...            
 4937    2023-10-24 19:00:00+02:00
 4938    2023-10-24 20:00:00+02:00
 4939    2023-10-24 21:00:00+02:00
 4940    2023-10-24 22:00:00+02:00
 4941    2023-10-24 23:00:00+02:00
 Name: DELIVERY_START, Length: 4942, dtype: object,
 array([19.44716423, 19.40788904, 19.51824614, ...,  9.105149  ,
         9.57211154,  9.8595796 ]))

In [35]:
d = {"DELIVERY_START" : delivery_start, "spot_id_delta" : y_pred.reshape(-1)}
y_pred = pd.DataFrame(data = d)

y_pred.to_csv("../data/output_y_test/y_result_indecision4.csv", index = False)

In [36]:
from collections import Counter

# classified_predictions contains the classified results, including -1 for unclassified instances # This should be the list you obtained in the previous step

# Use Counter to tally the occurrences of each class
class_counts = Counter(classified_predictions)

# Print the count of each class
print("Class 0 count:", class_counts[0])
print("Class 1 count:", class_counts[1])
print("Unclassified count:", class_counts[-1])


Class 0 count: 520
Class 1 count: 4141
Unclassified count: 281
